## Example: Polynomial with Exponentiation

**Theorem:** For all $x, y, z \in \mathbb{R}$ and $m, n \in \mathbb{Z}$, if $0 < z < 1$ and $0 < m < n$, then $2xy \cdot z^n \le (x^2 + y^2) \cdot z^m$.

This requires reasoning about exponential functions with integer exponents.

```lisp
(defun x^2+y^2 (x y) (+ (* x x) (* y y)))

(encapsulate ()
  (local (include-book "arithmetic-5/top" :dir :system))
  (defthm poly-of-expt-example
    (implies (and (real/rationalp x) (real/rationalp y) (real/rationalp z)
                  (integerp m) (integerp n)
                  (< 0 z) (< z 1) (< 0 m) (< m n))
             (<= (* 2 (expt z n) x y)
                 (* (expt z m) (x^2+y^2 x y))))
    :hints (("Goal"
             :smtlink-custom 
             (:functions ((expt :formals ((r real/rationalp)
                                          (i real/rationalp))
                                :returns ((ex real/rationalp))
                                :level 0))
              :hypotheses (((< (expt z n) (expt z m)))
                           ((< 0 (expt z m)))
                           ((< 0 (expt z n))))
              :int-to-rat t)))))
```

## Custom SMTLink Configuration

To use the custom exponentiation module:

```lisp
;; Define a custom configuration
(defun my-smtlink-expt-config ()
  (declare (xargs :guard t))
  (change-smtlink-config (default-smt-cnf)
                         :smt-module    "RewriteExpt"
                         :smt-class     "to_smt_w_expt"))

;; Attach the configuration
(defattach custom-smt-cnf my-smtlink-expt-config)
```

### What This Does:

1. **`:smt-module "RewriteExpt"`** - Uses `smtlink/z3_interface/RewriteExpt.py`
2. **`:smt-class "to_smt_w_expt"`** - The Python class that extends the default translator
3. **`defattach`** - Makes this the active configuration

## Understanding the Hints

### `:smtlink-custom` vs `:smtlink`

- **`:smtlink`** - Uses default (verified) SMTLink configuration
- **`:smtlink-custom`** - Uses attached custom configuration (separate trust tag)

### Treating `expt` as Uninterpreted

```lisp
:functions ((expt :formals ((r real/rationalp)
                            (i real/rationalp))
                  :returns ((ex real/rationalp))
                  :level 0))
```

- `:level 0` means Z3 doesn't expand `expt`'s definition
- Z3 only knows the type signature: `Real × Real → Real`

### Adding Hypotheses About `expt`

```lisp
:hypotheses (((< (expt z n) (expt z m)))
             ((< 0 (expt z m)))
             ((< 0 (expt z n))))
```

These lemmas about exponentials are:
1. Added to Z3's hypotheses
2. Proven as subgoals by ACL2 (using `arithmetic-5`)

## The RewriteExpt.py Module

Located at `smtlink/z3_interface/RewriteExpt.py`, this Python module:

1. Extends the default `ACL2_to_Z3.py` translator
2. Adds rewrite rules for `expt` that Z3 can use
3. Enables Z3 to reason about exponential relationships

### Trust Model

⚠️ **Important**: Custom modules are **not verified**!

- The user is responsible for ensuring correctness
- A separate trust tag is used: `(uses-smtlink-custom)`
- Bugs in custom modules can introduce unsoundness

## The `:int-to-rat` Hint

```lisp
:int-to-rat t
```

### Why It's Needed:

Z3 struggles with mixed integer/real arithmetic. This hint:

1. Treats all integers as rationals in Z3
2. Proves a more general statement (which implies the original)
3. Avoids Z3's non-linear integer arithmetic limitations

### When to Use:

- Theorems involving both `integerp` and `real/rationalp` variables
- When Z3 times out or fails on integer constraints

## Key Takeaways

1. **SMTLink is extensible** - Custom Python modules can add reasoning
2. **Uninterpreted functions** - `:level 0` hides implementation from Z3
3. **Hypotheses bridge the gap** - Tell Z3 facts about uninterpreted functions
4. **Trust carefully** - Custom modules bypass verification
5. **`:int-to-rat`** - Helps with mixed arithmetic

## Files Referenced

- `smtlink/examples/examples.lisp` - Example 2: `poly-of-expt-example`
- `smtlink/z3_interface/RewriteExpt.py` - Custom exponent handling
- `smtlink/z3_interface/ACL2_to_Z3.py` - Base translator class